In [1]:
import time 
import math
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
start_year = 2020
end_year = 2025
save_each_month = False
out_all_csv = "website_hk_private_cars_by_month.csv"
data_source = "https://webb-site.com/dbpub/veFR.asp"

# https://webb-site.com/dbpub/veFR.asp?y=2025&m=1&b=1&vc=1&sort=f1dn
params_template = {"b":"1", "vc":"1", "sort":"f1dn"}

In [3]:
# Standardize column names.
def normalize_cols(df):
    df = df.copy()
    df.columns = (df.columns
                 .str.strip()
                  .str.replace(r"\s+", " ", regex=True)
                  .str.replace("%", "pct")
                  .str.title()
                 )
    rename_map = {
        "Make": "Brand",
        "Brand Name": "Brand",
        "Brandname": "Brand",
        "Petrol": "Petrol",
        "Gasoline": "Petrol",
        "Electric": "Electric",
        "Ev": "Electric",
        "Total": "Total",
        "Grand Total": "Total",
        "Overall": "Total",
        "Hybrid": "Hybrid",
        "Diesel": "Diesel",
        "Lpg": "LPG",
        "Others": "Others"
    }
    
    for k, v in rename_map.items():
         if k in df.columns:
             df = df.rename(columns={k:v})
    return df

In [4]:
# Cleaning data
def coerce_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df

In [5]:
# Ensure the dataFrame has an "Others" column.
# If not , then compute it as "Total" -  sum of ["Petrol", "Electric", "Hybrid", "Diesel", "LPG"]
def compute_others(df):
    numeric_cols = [c for c in 
                    ["Petrol", "Electric", "Hybrid", "Diesel", "LPG", "Total"] if c in df.columns]
    df = coerce_numeric(df, numeric_cols)
    if "Others" not in df.columns:
        have = [c for c in ["Petrol", "Electric", "Hybrid", "Diesel", "LPG"] if c in df.columns]
        if "Total" in df.columns and have:
            df["Others"] = df["Total"] - df[have].sum(axis=1,skipna=True)
    return df  

In [6]:
from io import BytesIO
import traceback

In [7]:
def fetch_month(y, m, session, max_retries=3, pause=1.0):
    params = params_template | {"y":str(y), "m":str(m)}
    url = data_source
    for attempt in range (1,  max_retries+1):
        try:
            resp = session.get(url, params = params, timeout=20)
            if resp.status_code != 200:
                return None
            tables = pd.read_html(BytesIO(resp.content))
            if not tables :
                return None
            df = tables[0]
            if df.shape[1] <3 or df.shape[0] == 0:
                return None
            df = normalize_cols(df)
            if "Brand" in df.columns:
                df = df[df["Brand"].notna()].copy()
                # # Remove summary and footer rows, such as "Total", "All brands", "Fuel share %"
                pattern = r"(?i)^\s*(total|all\s*brands?|fuel\s*share\s*%)\s*$"
                mask_total = df["Brand"].astype(str).str.fullmatch(pattern)
                df = df[~mask_total].copy()
            else:
                return None
            # compute "Others" if the DataFrame doens't have it
            df = compute_others(df)
            # keep core columns that we foucus on 
            keep_cols = [c for c in
                         ["Brand","Petrol", "Electric", "Hybrid", "Diesel", "LPG", "Others", "Total"]
                         if c in df.columns]
            df = df[keep_cols].copy()   

            df = coerce_numeric(df, [c for c in keep_cols if c != "Brand"])
            df["Year"] = y
            df["Month"] = m
            return df
        except Exception as e:
            if attempt == max_retries:
                print(f"[{y}-{m:02d}] ERROR {type(e).__name__}: {e}")
                print(traceback.format_exc())
                return None
            time.sleep(pause * attempt) 
    return None

In [8]:
# single month test 
with requests.Session() as sess:
    sess.headers.update({"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    dfm = fetch_month(2020, 1, sess)
    print("test:", None if dfm is None else dfm.shape)
    if dfm is not None:
        print(dfm.head())

test: (40, 7)
     Brand  Petrol  Electric  Others  Total  Year  Month
0    TESLA     0.0      45.0     0.0   45.0  2020      1
1   NISSAN   101.0      25.0     0.0  126.0  2020      1
2     AUDI    83.0      22.0     0.0  105.0  2020      1
3  HYUNDAI    18.0       9.0     0.0   27.0  2020      1
4      KIA    36.0       7.0     0.0   43.0  2020      1


In [9]:
LATEST_Y, LATEST_M = 2025, 6

In [10]:
# collecting all months
all_frames = []
with requests.Session() as sess:
    sess.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    for y in range(2020, LATEST_Y+1 ):
        mmax = 12 if y < LATEST_Y else LATEST_M
        for m in range(1, mmax+1):
            dfm = fetch_month(y, m, sess)
            if dfm is None:
                print(f"[{y}-{m:02d}] no data")
            else:
                print(f"[{y}-{m:02d}] rows: {len(dfm)}")
                all_frames.append(dfm)
            time.sleep(0.6)

[2020-01] rows: 40
[2020-02] rows: 34
[2020-03] rows: 37
[2020-04] rows: 37
[2020-05] rows: 42
[2020-06] rows: 41
[2020-07] rows: 42
[2020-08] rows: 40
[2020-09] rows: 40
[2020-10] rows: 37
[2020-11] rows: 41
[2020-12] rows: 39
[2021-01] rows: 40
[2021-02] rows: 39
[2021-03] rows: 38
[2021-04] rows: 33
[2021-05] rows: 37
[2021-06] rows: 40
[2021-07] rows: 34
[2021-08] rows: 32
[2021-09] rows: 35
[2021-10] rows: 37
[2021-11] rows: 37
[2021-12] rows: 37
[2022-01] rows: 39
[2022-02] rows: 35
[2022-03] rows: 37
[2022-04] rows: 36
[2022-05] rows: 36
[2022-06] rows: 41
[2022-07] rows: 38
[2022-08] rows: 35
[2022-09] rows: 35
[2022-10] rows: 36
[2022-11] rows: 36
[2022-12] rows: 36
[2023-01] rows: 31
[2023-02] rows: 36
[2023-03] rows: 37
[2023-04] rows: 37
[2023-05] rows: 34
[2023-06] rows: 42
[2023-07] rows: 36
[2023-08] rows: 37
[2023-09] rows: 41
[2023-10] rows: 37
[2023-11] rows: 39
[2023-12] rows: 37
[2024-01] rows: 39
[2024-02] rows: 38
[2024-03] rows: 39
[2024-04] rows: 41
[2024-05] ro

In [11]:
if not all_frames:
    raise SystemExit("No data fetched.")

In [12]:
full = pd.concat(all_frames, ignore_index=True)
full = full.drop_duplicates(subset=["Year","Month","Brand"], keep="last") 
full = full.sort_values(["Year","Month","Brand"], ignore_index=True)
full.to_csv("webbsite_hk_private_cars_monthly_2020_2025.csv", index=False)

In [13]:
full["Brand"] = (full["Brand"].astype(str).str.strip()
                               .str.upper()
                               .str.replace(r"\s+", " ", regex=True))

In [14]:
ordered = ["Year","Month","Brand","Petrol","Electric","Hybrid","Diesel","LPG","Others","Total"]
full = full[[c for c in ordered if c in full.columns]]

In [15]:
full.to_csv(out_all_csv, index=False)
print("Saved:", out_all_csv)

Saved: website_hk_private_cars_by_month.csv


In [16]:
monthly = (full.groupby(["Year","Month"], as_index=False)
                .agg({c:"sum" for c in full.columns if c not in ["Year","Month","Brand"]}))

In [17]:
if {"Electric","Total"} <= set(monthly.columns):
    monthly["EV_share_%"] = monthly["Electric"]/monthly["Total"]*100

In [18]:
monthly.to_csv("webbsite_hk_private_cars_monthly_SUM_2020_2025.csv", index=False)
print("Saved: webbsite_hk_private_cars_monthly_SUM_2020_2025.csv")

Saved: webbsite_hk_private_cars_monthly_SUM_2020_2025.csv
